In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# [a]simply analysis

To exclude anomaly data, I make the graph about dataframe parameters and detect the anomaly data.

In [ ]:
df = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
df['excerpt'] = df['excerpt'].replace('\n','')
df['len'] = df.excerpt.apply(lambda x: len(x.split(" ")))

In [ ]:
plt.scatter(df.target.values,df.standard_error.values)
plt.show()

In [ ]:
df = df[df.target != 0]

In [ ]:
plt.scatter(df.target.values,df.standard_error.values)
plt.show()

In [ ]:
plt.scatter(df.target.values,df.len.values)
plt.show()

In [ ]:
df = df[df.len>130]
df = df[df.len<203]

In [ ]:
plt.scatter(df.target.values,df.len.values)
plt.show()

In [ ]:
plt.scatter(df.standard_error,df.len.values)
plt.show()

In [ ]:
df = df[df.standard_error<0.61]

In [ ]:
plt.scatter(df.standard_error,df.len.values)
plt.show()

In [ ]:
plt.scatter(df.target.values,df.standard_error.values)
plt.show()

In [ ]:
df = df[~((df.target <0) & (df.target>-2.0) & (df.standard_error>0.55))]

In [ ]:
plt.scatter(df.target.values,df.standard_error.values)
plt.show()

# Summary

In [ ]:
!mkdir /kaggle/tmp

In [ ]:
df = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
df['excerpt'] = df['excerpt'].replace('\n','')
df['len'] = df.excerpt.apply(lambda x: len(x.split(" ")))
df = df[df.target != 0]
df = df[df.len>130]
df = df[df.len<203]
df = df[df.standard_error<0.61]
df = df[~((df.target <0) & (df.target>-2.0) & (df.standard_error>0.55))]
df = df.reset_index(drop=True)
df.to_csv("/kaggle/tmp/train.csv")
df

# [b] text analysis

In [ ]:
import fasttext
train = pd.read_csv("/kaggle/tmp/train.csv")
f = open('data.txt', 'w')
f.write('\n'.join(train.excerpt.values))
f.close()
model = fasttext.train_unsupervised("data.txt", model='skipgram')

In [ ]:
train['text_mean'] = 0
train['text_var'] = 0

In [ ]:
embeddings = np.empty((0,210,100))
pad_arr = np.zeros((210,100))
mean = []
var = []
for i in range(len(train)):
    embedding = np.empty((0,100))
    if i%100==0:
        print(i)
    for text in train.excerpt.values[i].replace( '\n' , '' ).split(" "):
        embedding = np.append(embedding,model[text][None,:],axis=0)
    train.loc[i,'text_mean'] = embedding.mean()
    train.loc[i,'text_var'] = embedding.var()
    pad_arr[:embedding.shape[0],:] = embedding
    embeddings= np.append(embeddings,pad_arr[None,:,:],axis=0)

In [ ]:
plt.scatter(train.text_mean.values,train.text_var.values)

In [ ]:
train = train[train.text_var < 0.056]
train

In [ ]:
plt.scatter(train.text_mean.values,train.text_var.values)

In [ ]:
embeddings = embeddings[train.index]

In [ ]:
embeddings_flatten = embeddings.reshape(len(train),210*100)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
embeddings_pca = pca.fit_transform(embeddings_flatten)

In [ ]:
plt.scatter(embeddings_pca[:,0],embeddings_pca[:,1])

In [ ]:
train.to_csv("train.csv")